# Term Deposit Subscription Prediction

# Goal

Using the data collected from existing customers, build a model that will help the marketing
team identify potential customers who are relatively more likely to subscribe term deposit
and thus increase their hit ratio. 

## Attribute information:
  - Input variables 
    - Bank client data
        1. age: continuous feature
        2. job: type of job (management, technician, entrepreneur, blue-collar, etc.)
        3. marital: marital status (married, single, divorced)
        4. education: education level (primary, secondary, tertiary)
        5. default: has credit in default?
        6. housing: has housing loan?
        7. loan: has personal loan?
        8. balance in account
    - Related to previous contact
        9. contact: contact communication type
        10. month: last contact month of year
        11. day: last contact day of the month
        12. duration: last contact duration, in seconds*
    - Other attributes
        13. campaign: number of contacts performed during this campaign and for this
    - Client
        14. pdays: number of days that passed by after the client was last contacted from a
    - Previous campaign (-1 tells us the person has not been contacted or contact period is beyond 900 days)
        15. previous: number of times the client has been contacted before for the last
    - Campaign to subscribe term deposit
        16. poutcome: outcome of the previous marketing campaign
  - Output variable (desired target)
      17. Target: Tell us has the client subscribed a term deposit. (Yes, No)

# Import all necessary modules

In [7]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt 
import seaborn as sns
import pandas_profiling

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, confusion_matrix

from yellowbrick.classifier import ClassificationReport, ROCAUC

plt.style.use('ggplot')
pd.options.display.float_format = '{:,.2f}'.format
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

# Load and review data

In [8]:
df = pd.read_csv('bank-full.csv')

In [9]:
df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,Target
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


In [10]:
df.shape

(45211, 17)

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45211 entries, 0 to 45210
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   age        45211 non-null  int64 
 1   job        45211 non-null  object
 2   marital    45211 non-null  object
 3   education  45211 non-null  object
 4   default    45211 non-null  object
 5   balance    45211 non-null  int64 
 6   housing    45211 non-null  object
 7   loan       45211 non-null  object
 8   contact    45211 non-null  object
 9   day        45211 non-null  int64 
 10  month      45211 non-null  object
 11  duration   45211 non-null  int64 
 12  campaign   45211 non-null  int64 
 13  pdays      45211 non-null  int64 
 14  previous   45211 non-null  int64 
 15  poutcome   45211 non-null  object
 16  Target     45211 non-null  object
dtypes: int64(7), object(10)
memory usage: 5.9+ MB


In [12]:
df.describe()

,age,balance,day,duration,campaign,pdays,previous
count,"45,211.00","45,211.00","45,211.00","45,211.00","45,211.00","45,211.00","45,211.00"
mean,40.94,"1,362.27",15.81,258.16,2.76,40.20,0.58
std,10.62,"3,044.77",8.32,257.53,3.10,100.13,2.30
min,18.00,"-8,019.00",1.00,0.00,1.00,-1.00,0.00
25%,33.00,72.00,8.00,103.00,1.00,-1.00,0.00
50%,39.00,448.00,16.00,180.00,2.00,-1.00,0.00
75%,48.00,"1,428.00",21.00,319.00,3.00,-1.00,0.00
max,95.00,"102,127.00",31.00,"4,918.00",63.00,871.00,275.00


In [13]:
df.isnull().values.any()

False

# EDA

In [3]:
'''
EDA

1. Univariate analysis (12 marks)
a. Univariate analysis – data types and description of the independent attributes
which should include (name, meaning, range of values observed, central values
(mean and median), standard deviation and quartiles, analysis of the body of
distributions / tails, missing values, outliers.
b. Strategies to address the different data challenges such as data pollution, outlier’s
treatment and missing values treatment.
c. Please provide comments in jupyter notebook regarding the steps you take and
insights drawn from the plots.
'''

'\nEDA\n\n1. Univariate analysis (12 marks)\na. Univariate analysis – data types and description of the independent attributes\nwhich should include (name, meaning, range of values observed, central values\n(mean and median), standard deviation and quartiles, analysis of the body of\ndistributions / tails, missing values, outliers.\nb. Strategies to address the different data challenges such as data pollution, outlier’s\ntreatment and missing values treatment.\nc. Please provide comments in jupyter notebook regarding the steps you take and\ninsights drawn from the plots.\n'

In [4]:
'''
EDA

Multivariate analysis (8 marks)
a. Bi-variate analysis between the predictor variables and target column. Comment
on your findings in terms of their relationship and degree of relation if any.
Visualize the analysis using boxplots and pair plots, histograms or density curves.
Select the most appropriate attributes.
b. Please provide comments in jupyter notebook regarding the steps you take and
insights drawn from the plots
'''

'\nEDA\n\nMultivariate analysis (8 marks)\na. Bi-variate analysis between the predictor variables and target column. Comment\non your findings in terms of their relationship and degree of relation if any.\nVisualize the analysis using boxplots and pair plots, histograms or density curves.\nSelect the most appropriate attributes.\nb. Please provide comments in jupyter notebook regarding the steps you take and\ninsights drawn from the plots\n'

# Prepare the data

In [5]:
'''
Prepare the data

1. Ensure the attribute types are correct. If not, take appropriate actions.
2. Get the data model ready.
3. Transform the data i.e. scale / normalize if required
4. Create the training set and test set in ratio of 70:30
'''

'\nPrepare the data\n\n1. Ensure the attribute types are correct. If not, take appropriate actions.\n2. Get the data model ready.\n3. Transform the data i.e. scale / normalize if required\n4. Create the training set and test set in ratio of 70:30\n'

# Create the model

In [6]:
'''
Create the ensemble model

1. First create models using Logistic Regression and Decision Tree algorithm. Note the
model performance by using different matrices. Use confusion matrix to evaluate
class level metrics i.e. Precision/Recall. Also reflect the accuracy and F1 score of the
model. (10 marks)
2. Build the ensemble models (Bagging and Boosting) and note the model performance
by using different matrices. Use same metrics as in above model. (at least 3
algorithms) (15 marks)
3. Make a DataFrame to compare models and their metrics. Give conclusion regarding
the best algorithm and your reason behind it. (5 marks)
'''

'\nCreate the ensemble model\n\n1. First create models using Logistic Regression and Decision Tree algorithm. Note the\nmodel performance by using different matrices. Use confusion matrix to evaluate\nclass level metrics i.e. Precision/Recall. Also reflect the accuracy and F1 score of the\nmodel. (10 marks)\n2. Build the ensemble models (Bagging and Boosting) and note the model performance\nby using different matrices. Use same metrics as in above model. (at least 3\nalgorithms) (15 marks)\n3. Make a DataFrame to compare models and their metrics. Give conclusion regarding\nthe best algorithm and your reason behind it. (5 marks)\n'